In [1]:
import os

os.chdir('..')
%pwd

'e:\\DataScienceProjects\\car-price-prediction'

In [2]:
# Entity
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    processed_dataset_dir: Path
    embed_dim_file_path: str
    num_dim_file_path: str
    model_dir: Path
    batch_size: int
    epochs: int
    lr: float
    layers: List
    dropout: float

In [3]:
# Configuration Manager
from carPricePrediction.constants import *
from carPricePrediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([
            config.root_dir, 
            config.model_dir
        ])

        data_transformation_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            processed_dataset_dir = config.processed_dataset_dir,
            embed_dim_file_path = config.embed_dim_file_path,
            num_dim_file_path = config.num_dim_file_path,
            model_dir = config.model_dir,
            batch_size = params.batch_size,
            epochs = params.epochs,
            lr = params.lr,
            layers = params.layers,
            dropout = params.dropout       
        )

        return data_transformation_config

In [4]:
### Model class
import torch
import torch.nn as nn

class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_dim, layers, dout=0.5):
        super().__init__()
        # Create embedding layer
        self.embeds = nn.ModuleList([nn.Embedding(inp, out) for inp,out in embedding_dim])
        # Apply dropout (prevent overfitting)
        self.emb_drop = nn.Dropout(dout)
        # Apply batch normalization to numerical features
        self.bn_cont = nn.BatchNorm1d(n_cont)

        # Total dimension of embedding layers
        n_emb = sum((out for inp,out in embedding_dim))
        # Total input (embedding and continuous) 
        n_inp = n_emb + n_cont

        layerlist = []
        for i in layers:
            # Create Linear layer
            layerlist.append(nn.Linear(n_inp,i))
            # Add activation function
            layerlist.append(nn.ReLU(inplace=True))
            # Add batch normalization in the neurons
            layerlist.append(nn.BatchNorm1d(i))
            # Drop some neurons
            layerlist.append(nn.Dropout(dout))
            # The input for the next layer
            n_inp = i
        # Final layer - output layer
        layerlist.append(nn.Linear(layers[-1], out_dim))

        # Wrap the layers with Sequential
        self.layers = nn.Sequential(*layerlist)

    def forward(self, x_cat, x_cont): 
        # Create and concat embeddings
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x_cat = torch.cat(embeddings, 1)
        # Dropout
        x_cat = self.emb_drop(x_cat)
        # Batch normalization
        x_cont = self.bn_cont(x_cont)
        # Concat all features
        x = torch.cat([x_cat, x_cont], 1)
        # Return layer with input x
        layer = self.layers(x)
        
        return layer

In [5]:
import os
import pickle

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from carPricePrediction.components.feed_forward_nn import FeedForwardNN
from carPricePrediction.entity import ModelTrainerConfig
from carPricePrediction.logging.model_logger import ModelLogger

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def get_shuffled_batches(self, train_data, val_data, batch_size):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
        return train_loader, val_loader

    def train(self):
        # Choose device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load train and validation data
        train_filepath = os.path.join(self.config.processed_dataset_dir, "train.pth")
        train_data = torch.load(train_filepath)

        val_filepath = os.path.join(self.config.processed_dataset_dir, "val.pth")
        val_data = torch.load(val_filepath)

        # Load dimensions
        with open(self.config.num_dim_file_path, 'rb') as f:
            num_dim = pickle.load(f)

        with open(self.config.embed_dim_file_path, 'rb') as f:
            embedding_dim = pickle.load(f)
        
        # Set batches
        batch_size = self.config.batch_size
        train_loader, val_loader = self.get_shuffled_batches(train_data, val_data, batch_size)
        
        # Create model
        torch.manual_seed(100)
        model = FeedForwardNN(
            embedding_dim=embedding_dim, 
            n_cont=num_dim, 
            out_dim=1, 
            layers=self.config.layers, 
            dout=self.config.dropout
        ).to(device)

        # Choose loss funciton and optimizer
        loss_function = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=self.config.lr)

        # Create the model logger
        model_logger_file_path = os.path.join(self.config.model_dir, 'trainfile.log')
        modelLogger = ModelLogger(model_logger_file_path)

        # Set epochs
        num_epochs = self.config.epochs 

        # Train model
        for epoch in range(num_epochs):
            total_loss = 0
            model.train()
            for batch_idx, (cat_data, num_data, target) in enumerate(train_loader):
                cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
                optimizer.zero_grad()
                prediction = model(cat_data, num_data)
                loss = torch.sqrt(loss_function(prediction, target))
                total_loss += loss
                loss.backward()
                optimizer.step()
            total_loss /= len(train_loader)
            
            # Get validation loss
            val_loss = 0
            model.eval()
            with torch.no_grad():
                for cat_data, num_data, target in val_loader:
                    cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
                    prediction = model(cat_data, num_data)
                    val_loss += torch.sqrt(loss_function(prediction, target))
                val_loss /= len(val_loader)

            # Add to log file
            message = f'Epoch {epoch+1}/{num_epochs}, train_Loss: {total_loss}, val_Loss: {val_loss:.4f}'
            modelLogger.log_message(message)

        # Save the entire model
        model_file_path = os.path.join(self.config.model_dir, 'model.pth')
        torch.save(model, model_file_path)

        # Save model's state
        model_state_file_path = os.path.join(self.config.model_dir, 'model_state_dict.pth')
        torch.save(model.state_dict(), model_state_file_path)


In [6]:
# Model Trainer training pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-03-30 21:32:10,710: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-30 21:32:10,715: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-30 21:32:10,716: INFO: common: Created directory at: artifacts]
[2024-03-30 21:32:10,717: INFO: common: Created directory at: artifacts/model]
[2024-03-30 21:32:10,718: INFO: common: Created directory at: artifacts/model]


c:\Users\idale\anaconda3\envs\gpuenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-30 21:32:12,654: INFO: model_logger: Epoch 1/1000, train_Loss: 23704.708984375, val_Loss: 22507.0215]
[2024-03-30 21:32:12,856: INFO: model_logger: Epoch 2/1000, train_Loss: 23727.341796875, val_Loss: 22823.5742]
[2024-03-30 21:32:13,122: INFO: model_logger: Epoch 3/1000, train_Loss: 23681.849609375, val_Loss: 22600.8438]
[2024-03-30 21:32:13,321: INFO: model_logger: Epoch 4/1000, train_Loss: 23628.64453125, val_Loss: 22989.1367]
[2024-03-30 21:32:13,532: INFO: model_logger: Epoch 5/1000, train_Loss: 23704.0859375, val_Loss: 22714.9258]
[2024-03-30 21:32:13,844: INFO: model_logger: Epoch 6/1000, train_Loss: 23686.6796875, val_Loss: 23056.9258]
[2024-03-30 21:32:14,122: INFO: model_logger: Epoch 7/1000, train_Loss: 23609.1171875, val_Loss: 23187.7031]
[2024-03-30 21:32:14,357: INFO: model_logger: Epoch 8/1000, train_Loss: 23608.158203125, val_Loss: 22469.8105]
[2024-03-30 21:32:14,579: INFO: model_logger: Epoch 9/1000, train_Loss: 23607.236328125, val_Loss: 22583.6367]
[2024-03